## LangGraph로 에이전트 시스템 만들기
인터넷 검색과 코드 실행 도구를 결함한 에이전트 만들기

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

### State 설정

In [2]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import SystemMessage, HumanMessage


class State(TypedDict):
    messages: Annotated[list, add_messages]

### 도구 설정(웹, 파이썬 실행)

In [3]:
from langchain_tavily.tavily_search import TavilySearch
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import tool

web_search = TavilySearch(max_results=2)
repl = PythonREPL()

In [4]:
@tool
def python_repl(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. chart labels should be written in English.
    This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )


tools = [web_search, python_repl]
tool_node = ToolNode(tools)

### 그래프 정의

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [6]:
def agent(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

In [7]:
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"

In [8]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(State)

workflow.add_node("agent", agent)
workflow.add_node("tool", tool_node)

workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tool",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tool", "agent")

# Set up memory
memory = MemorySaver()

graph = workflow.compile(checkpointer=memory, interrupt_before=["tool"])

In [10]:
# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

테스트 

In [11]:
initial_input = {
    "messages": [HumanMessage(content="미국의 최근 5개년(~2023) GDP 차트를 그려줄래?")]
}
thread = {"configurable": {"thread_id": "13"}}
async for chunk in graph.astream(initial_input, thread, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values)
        print("\n\n")

Receiving update from node: 'agent'
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eKLkNLZXQZAQ1tZcfDN3Q7a4', 'function': {'arguments': '{"query":"US GDP data 2018 2019 2020 2021 2022 2023","time_range":"year","topic":"general"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 879, 'total_tokens': 924, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--41f99db0-8b98-4f61-b798-5afc949fe945-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'US GDP data 2018 2019 2020 2021 2022 2023', 'time_range': 'year', 'topic': 'general'}, 'id': 'call_eKLkNLZXQZAQ1tZcfDN3Q7a

In [12]:
async for chunk in graph.astream(None, thread, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values)
        print("\n\n")

Receiving update from node: 'tool'
{'messages': [ToolMessage(content="{'error': Exception('Error 502: Bad Gateway')}", name='tavily_search', id='feac3728-2454-4f97-aa40-7ac5f73a646a', tool_call_id='call_eKLkNLZXQZAQ1tZcfDN3Q7a4')]}



Receiving update from node: 'agent'
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dzVClADZqGaYTgQecuRdOk8w', 'function': {'arguments': '{"query":"US GDP 2018 2019 2020 2021 2022 2023","time_range":"year"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 946, 'total_tokens': 986, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ee5efcfd

- 다시 해볼것